In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from scipy.sparse import hstack

In [3]:
class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
train = pd.read_csv('data/train.csv').fillna(' ')
test = pd.read_csv('data/test.csv').fillna(' ')

In [4]:
train_text = train['comment_text']
test_text = test['comment_text']

In [ ]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 2), # use grid search on this and max_features
    max_features=12000)

train_word_features = word_vectorizer.fit_transform(train_text)
test_word_features = word_vectorizer.transform(test_text)

In [6]:
char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    stop_words='english',
    ngram_range=(2, 6), # use a grid search on this and max_features
    max_features=50000)

train_char_features = char_vectorizer.fit_transform(train_text)
test_char_features = char_vectorizer.transform(test_text)

In [7]:
train_features = hstack([train_char_features, train_word_features])
test_features = hstack([test_char_features, test_word_features])

In [8]:
scores = []

submission = pd.DataFrame.from_dict({'id': test['id']})

for class_name in class_names:
    train_target = train[class_name]
    classifier = LogisticRegression(solver='sag')

    cv_score = np.mean(cross_val_score(classifier, train_features, train_target, cv=3, scoring='roc_auc'))
    scores.append(cv_score)
    print('CV score for class {} is {}'.format(class_name, cv_score))

    classifier.fit(train_features, train_target)
    submission[class_name] = classifier.predict_proba(test_features)[:, 1]
    
print('Total CV score is {}'.format(np.mean(scores)))

CV score for class toxic is 0.9783206316415477
CV score for class severe_toxic is 0.9886033491873554
CV score for class obscene is 0.9902881267495672
CV score for class threat is 0.9898232456900827
CV score for class insult is 0.9826859095094159
CV score for class identity_hate is 0.9828028750652388
Total CV score is 0.9854206896405345


In [9]:
submission.to_csv('submission.csv', index=False)